In [120]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
pd.options.display.max_columns = None

In [105]:
train_pred_no_limit_table = pd.read_pickle("../data/train_pred_no_limit_table.pkl")
train_pred_2000_3000_table = pd.read_pickle("../data/train_pred_2000_3000_table.pkl")
train_pred_3000_4000_table = pd.read_pickle("../data/train_pred_3000_4000_table.pkl")

test_pred_no_limit_table = pd.read_pickle("../data/test_pred_no_limit_table.pkl")
test_pred_2000_3000_table = pd.read_pickle("../data/test_pred_2000_3000_table.pkl")
test_pred_3000_4000_table = pd.read_pickle("../data/test_pred_3000_4000_table.pkl")

In [21]:
test_pred_2000_3000_table

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5,radiant_synergy_wr,radiant_counter_wr,dire_synergy_wr,dire_counter_wr,rad_syn_mvp,rad_coun_mvp,dire_syn_mvp,dire_coun_mvp,counter_radiant_win,synergy_radiant_win,synergy_precision,counter_precision,synergy_radiant_mvp_win,counter_radiant_mvp_win,synergy_mvp_precision,counter_mvp_precision
0,5513575804,4623213746,True,1594777959,2511,2139,1,86,23,39,11,108,22,106,8,26,0.482,0.4892,0.501,0.5108,0.56,0.59,0.57,0.63,False,False,False,False,False,False,False,False
1,5513576209,4623212187,False,1594777979,2237,2530,45,26,12,29,87,14,17,64,72,99,0.491,0.4980,0.475,0.5020,0.54,0.57,0.53,0.61,False,True,False,True,True,False,False,True
2,5513577209,4623218161,True,1594778035,2661,2014,101,44,74,106,15,39,67,31,100,90,0.461,0.4700,0.512,0.5300,0.50,0.53,0.58,0.59,False,False,False,False,False,False,False,False
3,5513577815,4623214226,True,1594778061,2438,2812,46,85,25,42,40,45,84,96,48,21,0.495,0.4740,0.541,0.5260,0.59,0.56,0.57,0.60,False,False,False,False,True,False,True,False
4,5513578101,4623204258,False,1594778053,1931,2863,84,87,8,71,73,9,95,102,26,48,0.517,0.5024,0.513,0.4976,0.56,0.57,0.54,0.57,True,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5516010316,4625470045,True,1594857521,2447,2289,41,30,107,70,114,85,58,14,2,32,0.491,0.4920,0.487,0.5080,0.59,0.60,0.57,0.61,False,True,True,False,True,False,True,False
9996,5516010413,4625495756,True,1594857511,4019,2036,30,105,41,21,59,71,74,95,49,100,0.502,0.5040,0.495,0.4960,0.55,0.56,0.54,0.60,True,True,True,True,True,False,True,False
9997,5516011315,4625472625,False,1594857582,2598,2801,96,126,100,93,50,67,34,104,27,30,0.504,0.4944,0.505,0.5056,0.59,0.56,0.56,0.58,False,False,True,True,True,False,False,True
9998,5516011406,4625480100,False,1594857562,3127,2663,126,114,112,129,86,35,79,2,44,26,0.433,0.4724,0.485,0.5276,0.49,0.55,0.54,0.61,False,False,True,True,False,False,True,True


In [73]:
feature_set_columns = ["radiant_synergy_wr",
                       "radiant_counter_wr",
                       "dire_synergy_wr",
                       "dire_counter_wr",
                       "rad_syn_mvp",
                       "rad_coun_mvp",
                       "dire_syn_mvp",
                       "dire_coun_mvp"]

predictor_column = "radiant_win"

X_train = train_pred_2000_3000_table[feature_set_columns]
y_train = train_pred_2000_3000_table[predictor_column]

X_test = test_pred_2000_3000_table[feature_set_columns]
y_test = test_pred_2000_3000_table[predictor_column]


In [67]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [74]:
y_train = y_train.ravel()
y_test = y_test.ravel()

In [75]:


RandomForestRegModel = RandomForestClassifier()
RandomForestRegModel.fit(X_train,y_train)
y_train_pred = RandomForestRegModel.predict(X_train)

In [199]:
def random_forest_classifier(train, test, features, max_depth):
    
    # Set the predictor column
    predictor_column = "radiant_win"
    
    # Select only the features neccesary for the Train and test data set
    X_train = train[feature_set_columns]
    y_train = train[predictor_column]
    X_test = test[feature_set_columns]
    y_test = test[predictor_column]
    
    # Make the target data a series
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    
    # Instantiate the regression model
    RandomForestRegModel = RandomForestClassifier(max_depth)
    # Fit it on the training data
    RandomForestRegModel.fit(X_train,y_train)
    # Predict the y_train
    y_train_pred = RandomForestRegModel.predict(X_train)
    # Add it to the X_train dataframe and classify if accurate or not
    X_train = X_train.assign(y_train=y_train, y_train_pred=y_train_pred)
    X_train = X_train.assign(accuracy=np.where(X_train["y_train_pred"] == X_train["y_train"], True, False))
    
    # Use the classifier model to predict the test data
    y_test_pred = RandomForestRegModel.predict(X_test)
    X_test = X_test.assign(y_test=y_test, y_test_pred=y_test_pred)
    X_test = X_test.assign(accuracy=np.where(X_test["y_test_pred"] == X_test["y_test"], True, False))
    
    # Compute accuracy of the train set and test set
    train_accuracy = X_train["accuracy"].value_counts()[1] / len(X_train)
    test_accuracy = X_test["accuracy"].value_counts()[1] / len(X_test)
    
    # Print the result of the accuracy
    print(f"The accuracy of the train model RF Classifier is {train_accuracy}")
    print(f"The accuracy of the test model RF Classifier is {test_accuracy}")

In [162]:
def logistic_regression(train, test, feature_set_columns):
    
    # Set the predictor column
    predictor_column = "radiant_win"
    
    # Select only the features neccesary for the Train and test data set
    X_train = train[feature_set_columns]
    y_train = train[predictor_column]
    X_test = test[feature_set_columns]
    y_test = test[predictor_column]
    
    # Make the target data a series
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    
    # Instantiate the regression model
    LogisticModel = LogisticRegression()
    # Fit it on the training data
    LogisticModel.fit(X_train,y_train)
    # Predict the y_train
    y_train_pred = LogisticModel.predict(X_train)
    # Add it to the X_train dataframe and classify if accurate or not
    X_train = X_train.assign(y_train=y_train, y_train_pred=y_train_pred)
    X_train = X_train.assign(accuracy=np.where(X_train["y_train_pred"] == X_train["y_train"], True, False))
    
    # Use the classifier model to predict the test data
    y_test_pred = LogisticModel.predict(X_test)
    X_test = X_test.assign(y_test=y_test, y_test_pred=y_test_pred)
    X_test = X_test.assign(accuracy=np.where(X_test["y_test_pred"] == X_test["y_test"], True, False))
    
    # Compute accuracy of the train set and test set
    train_accuracy = X_train["accuracy"].value_counts()[1] / len(X_train)
    test_accuracy = X_test["accuracy"].value_counts()[1] / len(X_test)
    
    # Print the result of the accuracy
    print(f"The accuracy of the train model LR is {train_accuracy}")
    print(f"The accuracy of the test model LR is {test_accuracy}")

In [163]:
columns = ["radiant_synergy_wr",
           "radiant_counter_wr",
           "dire_synergy_wr",
           "dire_counter_wr",
           "rad_syn_mvp",
           "rad_coun_mvp",
           "dire_syn_mvp",
           "dire_coun_mvp"]

In [164]:
logistic_regression(train_pred_2000_3000_table, test_pred_2000_3000_table, columns)

The accuracy of the train model LR is 0.6094760947609477
The accuracy of the test model LR is 0.5779


In [165]:
logistic_regression(train_pred_3000_4000_table, test_pred_3000_4000_table, columns)

The accuracy of the train model LR is 0.60689
The accuracy of the test model LR is 0.5671


In [166]:
logistic_regression(train_pred_no_limit_table, test_pred_no_limit_table, columns)

The accuracy of the train model LR is 0.60807
The accuracy of the test model LR is 0.577057705770577


In [207]:
random_forest_classifier(train_pred_2000_3000_table, test_pred_2000_3000_table, columns, 1)

The accuracy of the train model RF Classifier is 0.8286182861828618
The accuracy of the test model RF Classifier is 0.525


In [208]:
random_forest_classifier(train_pred_3000_4000_table, test_pred_3000_4000_table, columns, 1)

The accuracy of the train model RF Classifier is 0.82764
The accuracy of the test model RF Classifier is 0.5137


In [209]:
random_forest_classifier(train_pred_no_limit_table, test_pred_no_limit_table, columns, 1)

The accuracy of the train model RF Classifier is 0.82999
The accuracy of the test model RF Classifier is 0.5164516451645165
